# Welcome to Justuse!

## Installation
Before we start, let's get the latest version

In [2]:
%%bash
python3 -m pip install justuse

  Using cached packaging-21.0-py3-none-any.whl (40 kB)
  Created wheel for justuse: filename=justuse-0.7.4-py3-none-any.whl size=144951 sha256=138c673b2c983aa3afd5dfcba7b45d37b5fe5fd467eb1744ace51b51fb564dfc
  Stored in directory: /home/thorsten/.cache/pip/wheels/d9/10/67/7436a55293bc60b77572182cee39d3a9d4dc9689ef6ac4fbd0
Successfully built justuse
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9


In [3]:
import use

DEBUG: urllib3.connectionpool: Starting new HTTPS connection (1): pypi.org:443
DEBUG: urllib3.connectionpool: https://pypi.org:443 "GET /pypi/justuse/json HTTP/1.1" 200 11098


In [4]:
use.__version__

'0.7.4'

## Basic Usage

Let's start with a simple case

In [5]:
use("math").cos(23)

INFO: use.main: case = (False, False, True, False)
INFO: use.buffet_old: result = <module 'math' (built-in)>


-0.5328330203333975

Here we loaded math but without adding it to the global namespace, so if you only need one thing from a module or package, you don't have to pollute your namespace.

Installing something could be inconvenient or unnecessary if something else is available - or we want to include some minimal functionality in our program and only fetch additional dependencies only under certain conditions.

The common approach would be something like

In [6]:
try:
    import some_big_package
except ImportError:
    some_big_package = None
if some_big_package:
    ...

which is unnecessarily cumbersome - couldn't we simply have a default like in so many other functions that is returned instead of raising an exception? Of course we can!

Here's a metaphor from *The Matrix*:
[![Matrix - Skill Upload](https://img.youtube.com/vi/w_8NsPQBdV0/0.jpg)](https://www.youtube.com/watch?v=w_8NsPQBdV0)

Imagine you want to streamline the user experience by distributing a very minimal, "free" but fully functional software to your end users which installs within seconds. Now, whenever the user wants to use a premium feature (or simply a feature that isn't generally required by the majority of users, therefor not included in the basic installation) the program could use() the packages and modules needed to realise the feature to download and install in the background while the user can still use other stuff, then trigger a callback when use() is done loading. The experience would be similar to playing an open world game which seamlessly downloads and loads new areas in the background on demand, without hiccup or loading screens. Or like Neo and Trinity - just get the skills to pilot a helicopter when you need them, right there on the spot. 

In [7]:
pkg = use("some_big_package", default=None)
if not pkg:
    print("I'm going to learn Ju-Jutsu?")

INFO: use.main: case = (False, False, False, False)
INFO: use.buffet_old: result = ImportError('No pkg installed named some_big_package and auto-installation not requested. Aborting.')


I'm going to learn Ju-Jutsu?


Even more concise (python >3.9):

In [8]:
if (pkg := use("pytest", default=None)):
    print("I know Kung Fu!")

INFO: use.main: case = (False, False, True, False)
INFO: use.buffet_old: result = <module 'pytest' from '/home/thorsten/.local/lib/python3.9/site-packages/pytest/__init__.py'>


I know Kung Fu!


It is also possible to lazily chain use() calls with defaults like

> math = use("numpy", default=None) or use("math")

which will evaluate from left to right and return whichever call returns something truthy (like a viable module) first.

One of the most practical use()s is making sure we imported the expected version of a certain package. This is especially important in research papers, notebooks and other publications because those often don't come with a `requirements.txt` and there is no way to make sure you are actually running the published code with the same versions as the author.
Also, if you pip-install something, it can happen that it upgrades a dependency, accidentally breaking code that requires the old version - with pip you *can't have more than one version installed*. With justuse any number of versions can be installed in parallel, without interfering with anything that was installed globally via pip, conda etc.

In [9]:
np = use("numpy", version="2022")

INFO: use.main: case = (True, False, True, False)
/home/thorsten/.local/lib/python3.9/site-packages/use/pimp.py:87: VersionWarning: numpy expected to be version 2022, but got 1.20.3 instead
  warn(UserMessage.version_warning(name, version, result_version), category=VersionWarning)
INFO: use.buffet_old: result = <module 'numpy' from '/home/thorsten/.local/lib/python3.9/site-packages/numpy/__init__.py'>


In [10]:
np.__version__

'1.20.3'

Here you see that even though we got a warning about the wrong version, we still get the requested package, just giving you a heads up about a possibly problematic situation without standing in the way.

Let's try another one!

In [11]:
pg = use("pygame")

INFO: use.main: case = (False, False, False, False)
INFO: use.buffet_old: result = ImportError('No pkg installed named pygame and auto-installation not requested. Aborting.')


ImportError: No pkg installed named pygame and auto-installation not requested. Aborting.

Well, bummer! We want to play with pygame, let's have pygame!

In [12]:
pg = use("pygame", modes=use.auto_install)

INFO: use.main: case = (False, False, False, True)
DEBUG: urllib3.connectionpool: Starting new HTTPS connection (1): pypi.org:443
DEBUG: urllib3.connectionpool: https://pypi.org:443 "GET /pypi/pygame/json HTTP/1.1" 200 221690
DEBUG: pip._internal.vcs.versioncontrol: Registered VCS backend: bzr
DEBUG: pip._internal.vcs.versioncontrol: Registered VCS backend: git
DEBUG: pip._internal.vcs.versioncontrol: Registered VCS backend: hg
DEBUG: pip._internal.vcs.versioncontrol: Registered VCS backend: svn


2.1.3.dev4


INFO: use.buffet_old: result = RuntimeWarning('Please specify version and hash for auto-installation of \'pygame\'.\nA webbrowser should open to the Snyk Advisor to check whether the package is vulnerable or malicious.\nIf you want to auto-install the latest version, try the following line to select all viable hashes:\nuse("pygame", version="2.1.3.dev4", modes=use.auto_install)')


RuntimeWarning: Please specify version and hash for auto-installation of 'pygame'.
A webbrowser should open to the Snyk Advisor to check whether the package is vulnerable or malicious.
If you want to auto-install the latest version, try the following line to select all viable hashes:
use("pygame", version="2.1.3.dev4", modes=use.auto_install)

Now we're getting somewhere! Hmm.. it says "To get some valuable insight on the health of this package, please check out https://snyk.io/advisor/python/pygame - see for yourself!

Let's look at this last line of the message.. hmm.. a dev version? Nah, let's pick the last stable version instead.

In [13]:
import platform; print(platform.system(), platform.release(), "\n", platform.python_version())

Linux 5.11.0-49-generic 
 3.9.5


In [14]:
use("pygame", version="2.1.2", modes=use.auto_install)

INFO: use.main: case = (True, False, False, True)
DEBUG: urllib3.connectionpool: Starting new HTTPS connection (1): pypi.org:443
DEBUG: urllib3.connectionpool: https://pypi.org:443 "GET /pypi/pygame/2.1.2/json HTTP/1.1" 200 221690
INFO: use.buffet_old: result = RuntimeWarning('Failed to auto-install \'pygame\' because hashes aren\'t specified.\n        A webbrowser should open with a list of available hashes for different platforms for you to pick."\n        If you want to use the package only on this platform, this should work:\n    use("pygame", version="2.1.2", hashes=\'T榮燰帛瓁䙶薮鼙䛔棙卮榙㿪茬䟐苙狋蹊\', modes=use.auto_install)')


T榮燰帛瓁䙶薮鼙䛔棙卮榙㿪茬䟐苙狋蹊


RuntimeWarning: Failed to auto-install 'pygame' because hashes aren't specified.
        A webbrowser should open with a list of available hashes for different platforms for you to pick."
        If you want to use the package only on this platform, this should work:
    use("pygame", version="2.1.2", hashes='T榮燰帛瓁䙶薮鼙䛔棙卮榙㿪茬䟐苙狋蹊', modes=use.auto_install)

Hey, did you see the browser tab that just opened? You can select all the platforms and python versions you want to support there and just copy & paste the snippet - let's get the one for python 3.8 and ubuntu I'm running here..

You wonder what those chinese looking characters are? Well. Normally, hexdigests look something like `d6c1c1c53a988b3daf44c1865d40f86de48665639bfbd5eea1317eb083638a3a` which is way too verbose on a normal line of code and since you shouldn't manually type those anyway but only copy&paste, we thought what the heck - let's use the japanese, ascii, chinese and korean alphabets (thus JACK) to encode those hashes as compact as we can.

Have a look at the last line of the Error message - let's try to copy&paste it..

In [16]:
pg = use("pygame", version="2.1.2", hashes='T榮燰帛瓁䙶薮鼙䛔棙卮榙㿪茬䟐苙狋蹊', modes=use.auto_install)

INFO: use.main: case = (True, True, False, True)
INFO: use.pimp: Attempting auto-installation of <87f20a6b5189ac48b057c687890624224236bfd009a43daf6d1d4aadf9aaa026>...
INFO: use.pimp: Downloading artifact from PyPI...
DEBUG: urllib3.connectionpool: Starting new HTTPS connection (1): files.pythonhosted.org:443
DEBUG: urllib3.connectionpool: https://files.pythonhosted.org:443 "GET /packages/4b/e2/e35260915e932f4e0ca905672ceabe1488940b6548d0ba62a3724f05390f/pygame-2.1.3.dev4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl HTTP/1.1" 200 21414738
INFO: use.pimp: Download successful.
INFO: use.pimp: Attempting to install..
INFO: use.pimp: Installation successful.
DEBUG: use.pimp: env_dir=/home/thorsten/.justuse-python/venv/pygame/2.1.2
DEBUG: use.pimp: site_dirs=[PosixPath('/home/thorsten/.justuse-python/venv/pygame/2.1.2/lib64/python3.9/site-packages'), PosixPath('/home/thorsten/.justuse-python/venv/pygame/2.1.2/lib/python3.9/site-packages')]
DEBUG: use.pimp:   - lib
DEBUG: use.pimp

pygame 2.1.3.dev4 (SDL 2.0.20, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


INFO: use.buffet_old: result = <module 'pygame' from '/home/thorsten/.justuse-python/venv/pygame/2.1.2/lib/python3.9/site-packages/pygame/__init__.py'>


Wow, did we just download, install and load the pygame package without leaving our own sweet code?? Yes, we did!

Furthermore, the package we installed is version and hash-pinned so we really only get what we asked for and nothing else.

There's a small problem though. Those hashes refer to very specific files and some of those packages may be written in C or even Fortran (like numpy) that are compiled for specific platforms.
If you happily develop code on Linux that uses something platform-specific (like numpy!) it will all work without problems - until you try to run your code on another platform. In this case, you need to specify all hashes for all platforms you want to run your code on.

Version- and hash-pinning is the most secure way to install a package. It will ensure that your code will always run as you expect it, but there's a drawback: there is no immediate and automatic way to update code without involving the user (yet). On one side, you won't ever accidentally break your stuff by updating something else, but you also won't benefit from automatic security patches. To fix this shortcoming, it might be feasible to build IDE-plugins that check and update these pins in the code or check some database for security patches every time an auto-installed package is imported - please contact us if you have ideas or better yet, code ;-)

## Use() modules from anywhere!
If you `import` some package or module, you're limited to the stuff you have in your current directory or below (but only if there is a `__init__.py` or if it's an implicit namespace package) and the things in your sys.path, which can be manipulated freely, making it very complicated to handle. Let's suppose we're in our test directory.

In [17]:
%cd ~/Desktop/sf_Dropbox/code/justuse/tests

[Errno 2] No such file or directory: '/home/thorsten/Desktop/sf_Dropbox/code/justuse/tests'
/media/sf_Dropbox_(Privat)/code/justuse/docs


the code we want to run is in justuse/docs and there is no `__init__.py` in between, so to get to run the code, we could put the src directory in sys.path - or we could use() a module directly!

In [18]:
mod = use(use.Path("../docs/demo.py"))

INFO: use.pimp: Path.cwd()=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs') package_name='' module_name='demo' module_path=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs/demo.py')


In [19]:
mod.foo()

Hello justuse-user!


Loading single modules doesn't sound like much, but especially while experimenting on jupyter, this can be used very effectively in conjunction with the reloading mode:

In [20]:
mod = use(use.Path("../docs/demo.py"), modes=use.reloading)

INFO: use.pimp: Path.cwd()=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs') package_name='' module_name='demo' module_path=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs/demo.py')


Now this module is loaded fresh whenever you modify and save the file, replacing the implementation behind the scene. This will work without any problems as long as you put functions in that module and if you access those functions via attribute-access (`mod.func()` **not** `func = mod.func; func()`).

If you can load modules from disk, why couldn't you load them from the web? Let's say you found an interesting github repo like https://github.com/amogorkon/justuse. Chances are, there's also a package on pypi you can pip-install, but maybe there's not. Maybe you're only interested in a single module from that repo/package, so you don't even want to install anything. Then you could download it from github, move the file manually into your folder and import it - sounds like a lot of trouble for a single module!
There has to be a better way! And there is - you can just use() web resources:

In [21]:
mod = use(use.URL("https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py"))

DEBUG: use.main: use-url: https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py
DEBUG: urllib3.connectionpool: Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG: urllib3.connectionpool: https://raw.githubusercontent.com:443 "GET /amogorkon/justuse/unstable/docs/demo.py HTTP/1.1" 200 60
/home/thorsten/.local/lib/python3.9/site-packages/use/main.py:399: NoValidationWarning: Attempting to import from the interwebs with no validation whatsoever!
To safely reproduce:
use(use.URL('https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py'), hash_algo=use.Hash.sha256, hash_value='59eff31bb220ce933ccc083b9306020ec25d19d43de30e5ad4341b355d4b48bf')
  warn(UserMessage.no_validation(url, hash_algo, this_hash), NoValidationWarning)
INFO: use.pimp: Path.cwd()=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs') package_name='' module_name='demo.py' module_path=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs/amogorkon/justuse/unsta

copy&paste that line from the exception to get that sweet hash..

In [23]:
mod = use(use.URL('https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py'), hash_algo=use.Hash.sha256, hash_value='59eff31bb220ce933ccc083b9306020ec25d19d43de30e5ad4341b355d4b48bf')

DEBUG: use.main: use-url: https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py
DEBUG: urllib3.connectionpool: Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG: urllib3.connectionpool: https://raw.githubusercontent.com:443 "GET /amogorkon/justuse/unstable/docs/demo.py HTTP/1.1" 200 60
INFO: use.pimp: Path.cwd()=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs') package_name='' module_name='demo.py' module_path=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs/amogorkon/justuse/unstable/docs/demo.py')


In [24]:
mod.foo()

Hello justuse-user!


Since the content of this file is now hash-pinned, it doesn't matter whether or not someone hacks github and changes the code - justuse will instantly notice before executing any code. You can even execute code directly from pastebin or any other untrusted, public platform - as long as you have the proper hash, you're safe.

## A word on circular imports
Everyone stumbles over a circular import once they try to build slightly more complex projects and it can get very ugly and overly frustrating to deal with those.

Let's suppose we have two modules A and B:

In [25]:
%cd ../docs
%ll

/media/sf_Dropbox_(Privat)/code/justuse/docs
total 63
-rwxrwx--- 1 root   2356 Mär 24 11:29  codeflow.md*
-rwxrwx--- 1 root   1326 Mär 24 11:29 'database schema.md'*
-rwxrwx--- 1 root     43 Feb 16 11:35  demo.py*
-rwxrwx--- 1 root     97 Feb 16 11:35  module_a.py*
-rwxrwx--- 1 root     47 Feb 16 11:35  module_b.py*
-rwxrwx--- 1 root     62 Feb 16 11:35  module_circular_a.py*
-rwxrwx--- 1 root     68 Feb 16 11:35  module_circular_b.py*
-rwxrwx--- 1 root 172380 Mär 29 18:04  Showcase.ipynb*


In [26]:
%less module_circular_a.py

print("Hello from A!")

import module_circular_b

foo = 23


In [27]:
%less module_circular_b.py


from module_circular_a import foo

print("Hello from B!", foo)


In [28]:
import module_circular_a

Hello from A!


ImportError: cannot import name 'foo' from partially initialized module 'module_circular_a' (most likely due to a circular import) (/media/sf_Dropbox_(Privat)/code/justuse/docs/module_circular_a.py)

In [29]:
%less module_a.py

import use

print("Hello from A!")

use(use.Path("module_b.py"), initial_globals={"foo": 23})


In [30]:
%less module_b.py

foo: int

print(f"Hello from B! foo={foo}")


In [31]:
modA = use(use.Path("module_a.py"))

INFO: use.pimp: Path.cwd()=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs') package_name='' module_name='module_a' module_path=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs/module_a.py')
INFO: use.pimp: Path.cwd()=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs') package_name='' module_name='module_b' module_path=PosixPath('/media/sf_Dropbox_(Privat)/code/justuse/docs/module_b.py')


Hello from A!
Hello from B! foo=23


## Aspects and decorators
If you have code like

```
def foo(x):
    return x ** 2
```
There are two ways to modify the behaviour of the code. Let's say you want to know which arguments were passed in and you want to print those arguments, so you can either add the print statement inside like
```
def foo(x):
    print(x)
    return x ** 2
```
or you can wrap the function with another function which gets called instead like

```
def decorator(x):
    print(x)
    return foo(x)
```
which you can also write as a function that takes a function as argument - a "higher order function" if you will - like
```
def decorator(func):
   def wrapper(*args, **kwargs):
       print(*args, **kwargs)
       return func(*args, **kwargs)
   return wrapper
foo = wrap(foo)
foo(x)
```
which looks a bit convoluted, so there is some syntactic sugar in python to make it look nicer:
```
@decorator
def foo(x):
    return x ** 2
```

There is nothing wrong with this approach, it's a plain and simple decorator, obvious reading the code. The only big problem is that single decorator is nice and simple but adding more quickly gets messy and complicated, not just because it gets hard to read and reason about but also because writing correct decorators is not as plain and simple as it may look like. For example, the *decorator* function above is bad because it will shadow the signature and docstring of the function it is wrapping. The second big problem is that it is quite messy to add decorators manually to code with lots of functions or methods. It can be not only a lot to write but also add visual noise, making it harder to read code (the opposite of what it was supposed to do!) while making it harder to keep [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself). The third big problem, no wait, amongst the chief problems of manually adding decorators is that it can be quite painful or impossible to deal with callables (functions, methods but also classes can be callable!) that originate in code you can't control, for example code that isn't even written in python but in C (like numpy).

Our solution: use() adds functionality to the modules you request and gives you an easy way to decorate *everything* in a single line:

In [32]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(func, args, kwargs)
        return func(*args, **kwargs)
    return wrapper

np = use("numpy")

INFO: use.main: case = (False, False, True, False)
INFO: use.buffet_old: result = <module 'numpy' from '/home/thorsten/.local/lib/python3.9/site-packages/numpy/__init__.py'>


In [34]:
np @  decorator

Please check your browser to select options and filters for aspects.


<module 'numpy' from '/home/thorsten/.local/lib/python3.9/site-packages/numpy/__init__.py'>

After waiting until the page is fully loaded (which can take a while..) we now can see all the thousands of callables inside numpy that we could apply our decorator with. This also allows us to play with different filters to make sure we only hit what we want. Now let's see what happens when we actually apply this decorator to all those callables.

In [39]:
use.apply_aspect(np, decorator)

<function __dir__ at 0x7fd364328ca0> () {}
<function __dir__ at 0x7fd364209820> () {}
<function __dir__ at 0x7fd3643e09d0> () {}
<function __dir__ at 0x7fd394d03940> () {}
<function __getattr__ at 0x7fd3642a4790> ('_ipython_canary_method_should_not_exist_',) {}
<function __getattr__ at 0x7fd364328c10> ('_ipython_canary_method_should_not_exist_',) {}
<function __getattr__ at 0x7fd364209790> ('_ipython_canary_method_should_not_exist_',) {}
<function __getattr__ at 0x7fd3643e0a60> ('_ipython_canary_method_should_not_exist_',) {}
<function __getattr__ at 0x7fd380d5adc0> ('_ipython_canary_method_should_not_exist_',) {}
<function __getattr__ at 0x7fd3642a4790> ('_ipython_display_',) {}
<function __getattr__ at 0x7fd364328c10> ('_ipython_display_',) {}
<function __getattr__ at 0x7fd364209790> ('_ipython_display_',) {}
<function __getattr__ at 0x7fd3643e0a60> ('_ipython_display_',) {}
<function __getattr__ at 0x7fd380d5adc0> ('_ipython_display_',) {}
<function __getattr__ at 0x7fd3642a4790> ('

<module 'numpy' from '/home/thorsten/.local/lib/python3.9/site-packages/numpy/__init__.py'>

We wrapped *all* functions in the numpy package with a decorator that lets us directly observe what happens when we call something without the need for an external debugger!

In [40]:
np.array([1,2,3])

<function array at 0x7fd3642a3550> ([1, 2, 3],) {}
<function array at 0x7fd364326940> ([1, 2, 3],) {}
<function array at 0x7fd364423d30> ([1, 2, 3],) {}
<function array at 0x7fd364403d30> ([1, 2, 3],) {}
<built-in function array> ([1, 2, 3],) {}
<function dtype at 0x7fd36429d670> <function dtype at 0x7fd36429d670> (<class 'int'>,) {}
<function dtype at 0x7fd36432e8b0> (<class 'int'>,) {}
<function dtype at 0x7fd3642f9820> (<class 'int'>,) {}
<function dtype at 0x7fd3643d1c10> (<class 'int'>,) {}
<class 'numpy.dtype'> (<class 'int'>,) {}


TypeError: issubclass() arg 2 must be a class or tuple of classes

If you aren't in a situation where you need to debug, log or control access (the classical applications of AOP) you might still want to use it in python - for type-checking/testing!
For example you can apply our favorite dynamic type checker library beartype quite easily like so:

In [ ]:
from beartype import beartype

np @ (use.isfunction, "", beartype)

Now all functions in numpy are beartype-checked! You can do the same with your own code, as long as you use() your modules.

If you want to learn more about aspect-oriented programming, check out https://en.wikipedia.org/wiki/Aspect-oriented_programming 

Even though applying decorators this way usually is super simple and nice, one word of caution: in python it sometimes is not clear at all *what* something is. If you do `foo(x)` foo could be a pure function, an object with a `__call__` method or you might be calling the `__init__` method of a class. While it is nice for the user to be oblivious about those details in general, in this case it can be quite confusing and frustrating because each case needs to be handled differently and using a check like isfunction might be catching the wrong things. So, be aware of these potential pitfalls before applying decorators blindly without checking what actually got hit.